In [1]:
import sys
sys.path.append('../scripts')

import experimenting
import event_library as el
import torch
from matplotlib import pyplot as plt
from experimenting.utils.visualization import plot_skeleton_2d, plot_skeleton_3d
from experimenting.utils.skeleton_helpers import Skeleton
import numpy as np
from experimenting.dataset.factory import Joints3DConstructor
import experimenting.utils.visualization as viz
import experimenting
from experimenting.utils import utilities

%load_ext autoreload
%autoreload 2

C:\Users\miracleyoo\Anaconda3\envs\dvs_pose\lib\site-packages\matplotlib_inline\config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [2]:
sadcore = experimenting.dataset.SadCore('test', root=r'J:\datasets\DVS\mmd\time_count_dataset', partition='cross-subject')
path = r'J:\datasets\DVS\mmd\experiments\exps\timecount_dhp19\exps_MargiposeEstimator\cross-subject\11-16-14-42_exp_resnet34_pretrained_False\checkpoints'
model = utilities.load_model(path, "MargiposeEstimator", core=sadcore).eval().double().cuda()

Loading training
Loading from ...  J:\datasets\DVS\mmd\experiments\exps\timecount_dhp19\exps_MargiposeEstimator\cross-subject\11-16-14-42_exp_resnet34_pretrained_False\checkpoints\epoch=00-val_loss=2.78.ckpt


In [3]:
factory = Joints3DConstructor()
factory.set_dataset_core(sadcore)
train, val, test = factory.get_datasets({'apply':{}}, {'apply':{}})

In [4]:
np.random.seed(100)
loader = iter(torch.utils.data.DataLoader(val, batch_size=1, shuffle=True))

b_x, b_y = next(loader)
print(f'Keys: {b_y.keys()}')

Keys: dict_keys(['xyz', 'skeleton', 'normalized_skeleton', 'z_ref', '2d_joints', 'M', 'camera', 'mask'])


In [5]:
counter=0
limits = [[-10, 10], [-10, 10], [0, 15]]
def plot_next(loader):
    b_x, b_y = next(loader)
    sk_label = Skeleton(b_y['xyz'][0])
    preds, outs = model(b_x.permute(0, -1, 1, 2).cuda())
    preds = preds.cpu()
    pred_sk = Skeleton(preds[0].detach().numpy()).denormalize(260, 346, camera=b_y['camera'][0], z_ref=b_y['z_ref'][0]).reproject_onto_world(b_y['M'][0])
    plot_skeleton_3d(sk_label, pred_sk, limits=limits)
    gt_joints = torch.stack([b_y['2d_joints'][0][:, 0], b_y['2d_joints'][0][:, 1]], 1)
    pred_joints = pred_sk.get_2d_points(260, 346, extrinsic_matrix=b_y['M'][0], intrinsic_matrix=b_y['camera'][0])
    plt.savefig(f'results/3D/{counter}.png')
    plt.close('all')

    pred_joints = np.stack([pred_joints[:, 0], pred_joints[:, 1]], 1)
    plot_skeleton_2d(b_x[0].squeeze(), gt_joints, pred_joints)
    plt.savefig(f'results/2D/{counter}.png')
    
    # plt.cla()
    plt.close('all')
    return sk_label, pred_sk

def to_float(x):
    if isinstance(x, np.ndarray):
        return x.astype(float)
    else:
        return x.float()


In [7]:
# plot_next(loader=loader)

In [ ]:
for i in range(100):
    plot_next(loader=loader)
    counter += 1

e:\GitHub\lifting_events_to_3d_hpe\notebooks\..\scripts\experimenting\utils\visualization.py:10: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax = Axes3D(fig)
No handles with labels found to put in legend.
e:\GitHub\lifting_events_to_3d_hpe\notebooks\..\scripts\experimenting\utils\visualization.py:10: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax = Axes3D(fig)
No hand